# Characterizing the general methylation landscape

In this notebook, I will characterize the general methylation landscape. This will provide context I need to understand the significance of differentially methylated loci I obtain with `methylKit`. To characterize CpG methylation, I will use individual samples, as well as a union BEDgraph that concatenates all sample information.

1. Concatenate coverage information
2. Characterize methylation for each CpG dinucleotide in individual samples and union BEDgraph
2. Determine genomic location of methylated, sparsely methylated, and unmethylated CpGs

## 0. Set working directory

In [1]:
!pwd

/Users/yaamini/Documents/project-oyster-oa/code/Haws


In [2]:
cd ../../analyses/

/Users/yaamini/Documents/project-oyster-oa/analyses


In [3]:
#!mkdir Haws_06-methylation-landscape

In [3]:
cd Haws_06-methylation-landscape/

/Users/yaamini/Documents/project-oyster-oa/analyses/Haws_06-methylation-landscape


In [4]:
#Install pandas for this notebook
import pandas as pd
print(pd.__version__)

0.18.1


## 1. Obtain sample BEDgraphs

In [6]:
#Download 5x bedgraphs
!wget -r \
--no-check-certificate --no-directories --no-parent --reject "index.html*" \
-P . \
-A "*5x.bedgraph" https://gannet.fish.washington.edu/spartina/project-oyster-oa/Haws/bismark-2/

--2021-05-17 20:50:07--  https://gannet.fish.washington.edu/spartina/project-oyster-oa/Haws/bismark-2/
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./index.html.tmp’

index.html.tmp          [ <=>                ] 168.69K  --.-KB/s    in 0.004s  

2021-05-17 20:50:12 (45.3 MB/s) - ‘./index.html.tmp’ saved [172743]

Loading robots.txt; please ignore errors.
--2021-05-17 20:50:12--  https://gannet.fish.washington.edu/robots.txt
Reusing existing connection to gannet.fish.washington.edu:443.
HTTP request sent, awaiting response... 404 Not Found
2021-05-17 20:50:12 ERROR 404: Not Found.

Removing ./index.html.tmp since it should be rejected.

--2021-05-17 20:50:12--  https://gannet.fish.washington.edu/spartina/pr

In [7]:
#Check directory for all files
!ls -lh

total 11482968
-rw-r--r--  1 yaamini  staff   240M Mar 11 02:25 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1 yaamini  staff   251M Mar 11 02:25 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1 yaamini  staff   227M Mar 11 02:25 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1 yaamini  staff   247M Mar 11 02:26 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1 yaamini  staff   191M Mar 11 02:26 zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1 yaamini  staff   258M Mar 11 02:26 zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1 yaamini  staff   227M Mar 11 02:26 zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1 yaamini  staff   244M Mar 11 02:27 zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1 yaamini  staff   235M Mar 11 02:27 zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
-rw-r--r--  1

In [8]:
#Obtain md5
!md5 *

MD5 (zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = f0dc26c38229b3640fa93fb29e1fa491
MD5 (zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = 0fd1e7003a0cb80de0e094cfdb8a7d0a
MD5 (zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = f4c8c3b70c40770c6d3376a2b7140925
MD5 (zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = 54cc1f3a915e03c34aa905fff5be2b63
MD5 (zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = cba3c994a0dc7502a64c5e0ae2c8727d
MD5 (zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = 9d54e2bd92b198b7ba4ab036d297b801
MD5 (zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = 54a28e3fd4ce60f6908ff11fc84e72c2
MD5 (zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = 650e074555739b4aac40df54abc79814
MD5 (zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = 2736303b8d17bce072892b08ac1ad978
MD5 (zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph) = 28ec1746f1cd1122eaea62bec434d38d


## 2. Concatenate 1x methylation information

I will use `unionBedGraphs` to concatenate information for all loci across samples.

In [4]:
bedtoolsDirectory = "/Users/Shared/bioinformatics/bedtools2/bin/"

In [10]:
!{bedtoolsDirectory}unionBedGraphs -h


Tool:    bedtools unionbedg (aka unionBedGraphs)
Version: v2.26.0
Summary: Combines multiple BedGraph files into a single file,
	 allowing coverage comparisons between them.

Usage:   bedtools unionbedg [OPTIONS] -i FILE1 FILE2 .. FILEn
	 Assumes that each BedGraph file is sorted by chrom/start 
	 and that the intervals in each are non-overlapping.

Options: 
	-header		Print a header line.
			(chrom/start/end + names of each file).

	-names		A list of names (one/file) to describe each file in -i.
			These names will be printed in the header line.

	-g		Use genome file to calculate empty regions.
			- STRING.

	-empty		Report empty regions (i.e., start/end intervals w/o
			values in all files).
			- Requires the '-g FILE' parameter.

	-filler TEXT	Use TEXT when representing intervals having no value.
			- Default is '0', but you can use 'N/A' or any text.

	-examples	Show detailed usage examples.



### 2a. Create a union BEDgraph

In [17]:
!rsync --archive --progress --verbose /Volumes/web/spartina/project-oyster-oa/Haws/bismark-2/*cov .

building file list ... 
24 files to consider
zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
   443999580 100%   48.14MB/s    0:00:08 (xfer#1, to-check=23/24)
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
   449661867 100%   45.31MB/s    0:00:09 (xfer#2, to-check=22/24)
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
   439244022 100%   46.69MB/s    0:00:08 (xfer#3, to-check=21/24)
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
   447737467 100%   44.12MB/s    0:00:09 (xfer#4, to-check=20/24)
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
   410215416 100%   43.46MB/s    0:00:09 (xfer#5, to-check=19/24)
zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
   451618171 100%   44.59MB/s    0:00:09 (xfer#6, to-check=18/24)
zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_Cp

In [18]:
!find *cov

zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov
zr3644_21_R1_val_

In [33]:
#Columns: chr, start, end, %meth, reads meth, reads unmeth
!head zr3644_9_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov

NC_001276.1	34	36	5.303030	7	125
NC_001276.1	123	125	1.315789	5	375
NC_001276.1	305	307	3.873239	11	273
NC_001276.1	433	435	2.153110	9	409
NC_001276.1	457	459	1.830664	8	429
NC_001276.1	482	484	0.477327	2	417
NC_001276.1	609	611	1.716738	8	458
NC_001276.1	781	783	1.434426	7	481
NC_001276.1	826	828	1.162791	5	425
NC_001276.1	951	953	3.603604	12	321


In [34]:
%%bash

for f in *.cov
do
    awk '{print $1"\t"$2"\t"$3"\t"$4}' ${f} \
    > $(basename ${f%..CpG_report.merged_CpG_evidence.cov})_1x.bedgraph
done

In [38]:
!find *1x.bedgraph

zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_21_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_22_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_23_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_24_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_2_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.bedgraph
zr3644_3_R1_val_

In [39]:
%%bash

for f in *1x.bedgraph
do
    /Users/Shared/bioinformatics/bedtools2/bin/sortBed \
    -i ${f} \
    > $(basename ${f%_1x.bedgraph})_1x.sort.bedgraph
done

In [40]:
!ls *1x.sort.bedgraph

zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_21_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_22_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_23_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.bedgraph
zr3644_24_R1_val_1_val_1_val_1_bismark_bt2_pe_1x.sort.be

In [41]:
#Create union BEDgraph from sorted files
#Include a header
#Use N/A when there is no data for a CpG in a sample
#Define sample IDs
#Use sorted bedgraphs
#Save output
!{bedtoolsDirectory}unionBedGraphs \
-header \
-filler N/A \
-names 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 \
-i \
*1x.sort.bedgraph \
> union_1x.bedgraph

In [42]:
#Check output
!head union_1x.bedgraph
!wc -l union_1x.bedgraph

chrom	start	end	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24
NC_001276.1	34	36	3.092784	1.162791	3.061224	3.401361	3.614458	2.659574	4.375000	4.347826	4.054054	3.603604	2.343750	3.468208	2.083333	2.453988	6.250000	5.479452	9.615385	6.862745	5.217391	1.785714	2.150538	3.389831	2.380952	5.303030
NC_001276.1	123	125	0.851064	0.425532	0.826446	0.755668	1.562500	0.821355	0.493827	1.100917	0.819672	0.387597	2.006689	0.956938	0.000000	0.512821	1.115242	1.020408	0.378788	0.396825	1.689189	0.727273	1.176471	0.845666	1.287554	1.315789
NC_001276.1	305	307	3.314917	1.081081	3.225806	1.955307	6.578947	2.739726	1.973684	2.403846	2.427184	1.777778	2.525253	3.669725	4.848485	3.846154	3.255814	5.166052	2.843602	5.687204	2.517986	1.621622	1.630435	4.450262	4.040404	3.873239
NC_001276.1	433	435	1.986755	1.167315	1.133144	0.569260	0.881057	1.798561	1.212121	1.658375	1.292407	1.273885	0.284900	1.541426	0.743494	0.649351	1.562500	1.834862	1.277955	2.473498	0.997506	1.506024	1.683502	1.6806

## 3. Concatenate 5x methylation information

### 3a. Create a union BEDgraph

In [11]:
%%bash

for f in *5x.bedgraph
do
/Users/Shared/bioinformatics/bedtools2/bin/sortBed \
-i ${f} \
> $(basename ${f%_5x.bedgraph})_5x.sort.bedgraph
done

In [12]:
!ls *sort*

zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_21_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_22_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_23_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.sort.bedgraph
zr3644_24_R1_val_1_val_1_val_1_bismark_bt

In [13]:
#Create union BEDgraph from sorted files
#Include a header
#Use N/A when there is no data for a CpG in a sample
#Define sample IDs
#Use sorted bedgraphs
#Save output
!{bedtoolsDirectory}unionBedGraphs \
-header \
-filler N/A \
-names 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 \
-i \
*5x.sort.bedgraph \
> union_5x.bedgraph

In [14]:
#Check output
!head union_5x.bedgraph
!wc -l union_5x.bedgraph

chrom	start	end	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24
NC_001276.1	34	36	3.092784	1.162791	3.061224	3.401361	3.614458	2.659574	4.375000	4.347826	4.054054	3.603604	2.343750	3.468208	2.083333	2.453988	6.250000	5.479452	9.615385	6.862745	5.217391	1.785714	2.150538	3.389831	2.380952	5.303030
NC_001276.1	123	125	0.851064	0.425532	0.826446	0.755668	1.562500	0.821355	0.493827	1.100917	0.819672	0.387597	2.006689	0.956938	0.000000	0.512821	1.115242	1.020408	0.378788	0.396825	1.689189	0.727273	1.176471	0.845666	1.287554	1.315789
NC_001276.1	305	307	3.314917	1.081081	3.225806	1.955307	6.578947	2.739726	1.973684	2.403846	2.427184	1.777778	2.525253	3.669725	4.848485	3.846154	3.255814	5.166052	2.843602	5.687204	2.517986	1.621622	1.630435	4.450262	4.040404	3.873239
NC_001276.1	433	435	1.986755	1.167315	1.133144	0.569260	0.881057	1.798561	1.212121	1.658375	1.292407	1.273885	0.284900	1.541426	0.743494	0.649351	1.562500	1.834862	1.277955	2.473498	0.997506	1.506024	1.683502	1.6806

### 3b. Manipulate with `pandas`

In [5]:
#Import union data into pandas
#Check head
df = pd.read_table("union_5x.bedgraph")
df.head(5)

,chrom,start,end,1,2,3,4,5,6,7,...,15,16,17,18,19,20,21,22,23,24
0,NC_001276.1,34,36,3.092784,1.162791,3.061224,3.401361,3.614458,2.659574,4.375000,...,6.250000,5.479452,9.615385,6.862745,5.217391,1.785714,2.150538,3.389831,2.380952,5.303030
1,NC_001276.1,123,125,0.851064,0.425532,0.826446,0.755668,1.562500,0.821355,0.493827,...,1.115242,1.020408,0.378788,0.396825,1.689189,0.727273,1.176471,0.845666,1.287554,1.315789
2,NC_001276.1,305,307,3.314917,1.081081,3.225806,1.955307,6.578947,2.739726,1.973684,...,3.255814,5.166052,2.843602,5.687204,2.517986,1.621622,1.630435,4.450262,4.040404,3.873239
3,NC_001276.1,433,435,1.986755,1.167315,1.133144,0.569260,0.881057,1.798561,1.212121,...,1.562500,1.834862,1.277955,2.473498,0.997506,1.506024,1.683502,1.680672,2.135231,2.153110
4,NC_001276.1,457,459,1.538462,2.158273,0.529101,0.172117,1.195219,1.487603,0.560748,...,0.857143,0.643777,1.159420,0.619195,2.142857,1.983003,1.238390,1.374046,2.317881,1.830664


In [6]:
#Average all samples for total genome methylation information and save as a new column
#NA are not included in averages
#Check output
df['total'] = df[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']].mean(axis=1)
df.tail(10)

,chrom,start,end,1,2,3,4,5,6,7,...,16,17,18,19,20,21,22,23,24,total
10497310,NW_022994998.1,54647,54649,0.0,0.0,9.090909,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.378788
10497311,NW_022994998.1,54770,54772,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000
10497312,NW_022994998.1,54834,54836,0.0,0.0,0.000000,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.000000
10497313,NW_022994998.1,54843,54845,0.0,NaN,0.000000,NaN,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.954545
10497314,NW_022994998.1,54860,54862,0.0,NaN,0.000000,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.000000
10497315,NW_022994998.1,54872,54874,0.0,NaN,0.000000,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.000000
10497316,NW_022994998.1,54934,54936,0.0,NaN,0.000000,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.000000
10497317,NW_022994998.1,54949,54951,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.000000
10497318,NW_022994998.1,54953,54955,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.000000
10497319,NW_022994998.1,54958,54960,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.000000


In [17]:
#Save dataframe in a tabular format and include N/As. Do not include quotes.
df.to_csv("union-averages.bedgraph", sep = "\t", na_rep = "N/A", quoting = 3)

In [18]:
#Check pandas manipulations
!head union-averages.bedgraph

	chrom	start	end	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	total
0	NC_001276.1	34	36	3.092784	1.162791	3.061224	3.401361	3.6144580000000004	2.659574	4.375	4.347826	4.054054	3.6036040000000003	2.34375	3.468208	2.083333	2.4539880000000003	6.25	5.479451999999999	9.615385	6.862744999999999	5.217391	1.785714	2.150538	3.3898309999999996	2.380952	5.303030000000001	3.8398747083333338
1	NC_001276.1	123	125	0.8510639999999999	0.42553199999999997	0.8264459999999999	0.755668	1.5625	0.821355	0.493827	1.100917	0.819672	0.38759699999999997	2.0066889999999997	0.956938	0.0	0.512821	1.115242	1.020408	0.378788	0.396825	1.689189	0.727273	1.1764709999999998	0.845666	1.287554	1.3157889999999999	0.894759625
2	NC_001276.1	305	307	3.314917	1.081081	3.225806	1.955307	6.578947	2.7397259999999997	1.973684	2.4038459999999997	2.427184	1.7777779999999999	2.5252529999999997	3.669725	4.848485	3.8461540000000003	3.255814	5.166052	2.843602	5.687204	2.517986	1.621622	1.630435	4.450262	4.040404	3.8732

In [19]:
#Remove header
#Keep chr, start, end, and the average
#Save output
! tail -n+2 union-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $2, $3, $4, $29}' \
> zr3644_union-averages_5x.bedgraph

In [20]:
#Check output: chr, start, end, average %meth
!head zr3644_union-averages_5x.bedgraph
!wc -l zr3644_union-averages_5x.bedgraph

NC_001276.1	34	36	3.8398747083333338
NC_001276.1	123	125	0.894759625
NC_001276.1	305	307	3.2272713749999995
NC_001276.1	433	435	1.3957046250000003
NC_001276.1	457	459	1.2465982500000001
NC_001276.1	482	484	1.2571130416666667
NC_001276.1	609	611	2.1339244166666664
NC_001276.1	781	783	2.1547547083333334
NC_001276.1	826	828	1.3927336666666665
NC_001276.1	951	953	3.3334975000000004
 10497320 zr3644_union-averages_5x.bedgraph


In [7]:
#Average all diploid samples and save as a new column
#Average all triploid samples and save as a new column
#NA are not included in averages
#Check output
df['diploid'] = df[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']].mean(axis=1)
df['triploid'] = df[['13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']].mean(axis=1)
df.tail(10)

,chrom,start,end,1,2,3,4,5,6,7,...,18,19,20,21,22,23,24,total,diploid,triploid
10497310,NW_022994998.1,54647,54649,0.0,0.0,9.090909,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.378788,0.757576,0.0
10497311,NW_022994998.1,54770,54772,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,0.000000,0.0
10497312,NW_022994998.1,54834,54836,0.0,0.0,0.000000,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.000000,0.000000,0.0
10497313,NW_022994998.1,54843,54845,0.0,NaN,0.000000,NaN,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.954545,1.909091,0.0
10497314,NW_022994998.1,54860,54862,0.0,NaN,0.000000,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.000000,0.000000,0.0
10497315,NW_022994998.1,54872,54874,0.0,NaN,0.000000,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.000000,0.000000,0.0
10497316,NW_022994998.1,54934,54936,0.0,NaN,0.000000,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.000000,0.000000,0.0
10497317,NW_022994998.1,54949,54951,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,0.000000,0.0
10497318,NW_022994998.1,54953,54955,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,0.000000,0.0
10497319,NW_022994998.1,54958,54960,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,0.000000,0.0


In [8]:
#Save dataframe in a tabular format and include N/As. Do not include quotes.
df.to_csv("union-averages-ploidy.bedgraph", sep = "\t", na_rep = "N/A", quoting = 3)

In [9]:
#Check pandas manipulations
!head union-averages-ploidy.bedgraph

	chrom	start	end	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	total	diploid	triploid
0	NC_001276.1	34	36	3.092784	1.162791	3.061224	3.401361	3.6144580000000004	2.659574	4.375	4.347826	4.054054	3.6036040000000003	2.34375	3.468208	2.083333	2.4539880000000003	6.25	5.479451999999999	9.615385	6.862744999999999	5.217391	1.785714	2.150538	3.3898309999999996	2.380952	5.303030000000001	3.8398747083333338	3.265386166666667	4.41436325
1	NC_001276.1	123	125	0.8510639999999999	0.42553199999999997	0.8264459999999999	0.755668	1.5625	0.821355	0.493827	1.100917	0.819672	0.38759699999999997	2.0066889999999997	0.956938	0.0	0.512821	1.115242	1.020408	0.378788	0.396825	1.689189	0.727273	1.1764709999999998	0.845666	1.287554	1.3157889999999999	0.894759625	0.9173504166666665	0.8721688333333333
2	NC_001276.1	305	307	3.314917	1.081081	3.225806	1.955307	6.578947	2.7397259999999997	1.973684	2.4038459999999997	2.427184	1.7777779999999999	2.5252529999999997	3.669725	4.848485	3.8461540000000003	3.2

In [12]:
#Remove header
#Keep chr, start, end, and the average
#Save output
! tail -n+2 union-averages-ploidy.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $30, $31}' \
> ploidy-methylation-averages.bedgraph

In [13]:
#Check output: chr, start, end, average %meth
!head ploidy-methylation-averages.bedgraph
!wc -l ploidy-methylation-averages.bedgraph

3.265386166666667	4.41436325
0.9173504166666665	0.8721688333333333
2.8061045	3.64843825
1.2332671666666666	1.5581420833333333
1.10236375	1.3908327499999997
1.1548041666666666	1.3594219166666663
1.8039629166666664	2.4638859166666665
1.9781402500000003	2.3313691666666667
1.4114300000000002	1.3740373333333336
3.294818	3.372177
 10497320 ploidy-methylation-averages.bedgraph


## 4. Concatenate coverage information

As methylation detection relies on coverage, I want to understand potential coverage differences between diploids and triploids. This requires concatenating coverage information across samples.

### 4a. Format files

In [19]:
%%bash

for f in *cov
do
  awk '{print $1"\t"$2"\t"$3"\t"$5+$6}' ${f} \
  > ${f}.txt
done

In [20]:
!find *cov.txt
!head *cov.txt

zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt
zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_repo

### 4b. Create union coverage file

In [21]:
%%bash

for f in *cov.txt
do
/Users/Shared/bioinformatics/bedtools2/bin/sortBed \
-i ${f} \
> $(basename ${f%_cov.txt})_5x.sort.cov
done

In [22]:
!ls *sort*

zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe..CpG_report.merged_CpG_evidence.cov.txt_5x.sort.cov
zr3644_1_R1_val_1_va

In [25]:
#Create union BEDgraph from sorted files
#Include a header
#Use N/A when there is no data for a CpG in a sample
#Define sample IDs
#Use sorted bedgraphs
#Save output
!{bedtoolsDirectory}unionBedGraphs \
-header \
-filler N/A \
-names 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 \
-i \
*5x.sort.cov \
> union_5x.cov

In [26]:
#Check output
!head union_5x.cov
!wc -l union_5x.cov

chrom	start	end	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24
NC_001276.1	34	36	97	86	98	147	83	188	160	207	222	111	128	173	96	163	112	146	104	102	115	112	93	177	84	132
NC_001276.1	123	125	235	235	242	397	192	487	405	545	488	258	299	418	219	390	269	392	264	252	296	275	255	473	233	380
NC_001276.1	305	307	181	185	186	358	152	365	304	416	412	225	198	327	165	338	215	271	211	211	278	185	184	382	198	284
NC_001276.1	433	435	302	257	353	527	227	556	495	603	619	314	351	519	269	462	320	436	313	283	401	332	297	595	281	418
NC_001276.1	457	459	325	278	378	581	251	605	535	644	653	338	387	549	285	520	350	466	345	323	420	353	323	655	302	437
NC_001276.1	482	484	313	289	343	538	243	567	483	604	602	280	348	509	274	510	315	414	334	310	391	348	329	595	288	419
NC_001276.1	609	611	338	324	371	629	287	668	533	676	694	346	401	571	339	584	360	494	381	354	426	387	375	680	314	466
NC_001276.1	781	783	343	308	309	580	268	624	517	666	683	364	422	528	313	483	356	489	373	305	419	362	330	641	285	488
NC

### 4c. Manipulate with `pandas`

In [27]:
#Import union data into pandas
#Check head
df = pd.read_table("union_5x.cov")
df.head(5)

,chrom,start,end,1,2,3,4,5,6,7,...,15,16,17,18,19,20,21,22,23,24
0,NC_001276.1,34,36,97.0,86.0,98.0,147.0,83.0,188.0,160.0,...,112.0,146.0,104.0,102.0,115.0,112.0,93.0,177.0,84.0,132.0
1,NC_001276.1,123,125,235.0,235.0,242.0,397.0,192.0,487.0,405.0,...,269.0,392.0,264.0,252.0,296.0,275.0,255.0,473.0,233.0,380.0
2,NC_001276.1,305,307,181.0,185.0,186.0,358.0,152.0,365.0,304.0,...,215.0,271.0,211.0,211.0,278.0,185.0,184.0,382.0,198.0,284.0
3,NC_001276.1,433,435,302.0,257.0,353.0,527.0,227.0,556.0,495.0,...,320.0,436.0,313.0,283.0,401.0,332.0,297.0,595.0,281.0,418.0
4,NC_001276.1,457,459,325.0,278.0,378.0,581.0,251.0,605.0,535.0,...,350.0,466.0,345.0,323.0,420.0,353.0,323.0,655.0,302.0,437.0


In [28]:
#Average all diploid samples and save as a new column
#Average all triploid samples and save as a new column
#NA are not included in averages
#Check output
df['diploid'] = df[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']].mean(axis=1)
df['triploid'] = df[['13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']].mean(axis=1)
df.tail(10)

,chrom,start,end,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,24,diploid,triploid
12794064,NW_022994998.1,54834,54836,7.0,5.0,13.0,3.0,8.0,7.0,10.0,...,6.0,5.0,7.0,8.0,6.0,14.0,4.0,2.0,8.083333,7.000000
12794065,NW_022994998.1,54843,54845,7.0,4.0,12.0,4.0,8.0,7.0,10.0,...,6.0,5.0,7.0,8.0,6.0,12.0,4.0,3.0,8.000000,6.833333
12794066,NW_022994998.1,54860,54862,5.0,4.0,12.0,3.0,7.0,6.0,11.0,...,6.0,6.0,7.0,6.0,6.0,14.0,4.0,3.0,7.416667,6.833333
12794067,NW_022994998.1,54872,54874,5.0,4.0,11.0,2.0,7.0,6.0,8.0,...,6.0,6.0,6.0,6.0,6.0,13.0,4.0,3.0,6.583333,6.333333
12794068,NW_022994998.1,54934,54936,6.0,2.0,5.0,2.0,3.0,7.0,3.0,...,3.0,4.0,2.0,2.0,2.0,5.0,5.0,1.0,4.181818,2.916667
12794069,NW_022994998.1,54949,54951,4.0,2.0,3.0,1.0,2.0,6.0,2.0,...,1.0,4.0,2.0,1.0,2.0,3.0,5.0,1.0,2.909091,2.500000
12794070,NW_022994998.1,54953,54955,2.0,2.0,3.0,1.0,2.0,6.0,2.0,...,1.0,3.0,2.0,1.0,2.0,3.0,5.0,1.0,2.636364,2.363636
12794071,NW_022994998.1,54958,54960,2.0,2.0,3.0,1.0,2.0,6.0,2.0,...,1.0,3.0,2.0,1.0,2.0,3.0,5.0,1.0,2.636364,2.272727
12794072,NW_022994998.1,55001,55003,NaN,1.0,1.0,1.0,1.0,4.0,NaN,...,NaN,1.0,NaN,NaN,NaN,3.0,2.0,NaN,1.600000,1.600000
12794073,NW_022994998.1,55026,55028,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000


In [29]:
#Save dataframe in a tabular format and include N/As. Do not include quotes.
df.to_csv("union-cov-ploidy.bedgraph", sep = "\t", na_rep = "N/A", quoting = 3)

In [30]:
#Check pandas manipulations
!head union-cov-ploidy.bedgraph

	chrom	start	end	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	diploid	triploid
0	NC_001276.1	34	36	97.0	86.0	98.0	147.0	83.0	188.0	160.0	207.0	222.0	111.0	128.0	173.0	96.0	163.0	112.0	146.0	104.0	102.0	115.0	112.0	93.0	177.0	84.0	132.0	141.66666666666666	119.66666666666667
1	NC_001276.1	123	125	235.0	235.0	242.0	397.0	192.0	487.0	405.0	545.0	488.0	258.0	299.0	418.0	219.0	390.0	269.0	392.0	264.0	252.0	296.0	275.0	255.0	473.0	233.0	380.0	350.0833333333333	308.1666666666667
2	NC_001276.1	305	307	181.0	185.0	186.0	358.0	152.0	365.0	304.0	416.0	412.0	225.0	198.0	327.0	165.0	338.0	215.0	271.0	211.0	211.0	278.0	185.0	184.0	382.0	198.0	284.0	275.75	243.5
3	NC_001276.1	433	435	302.0	257.0	353.0	527.0	227.0	556.0	495.0	603.0	619.0	314.0	351.0	519.0	269.0	462.0	320.0	436.0	313.0	283.0	401.0	332.0	297.0	595.0	281.0	418.0	426.9166666666667	367.25
4	NC_001276.1	457	459	325.0	278.0	378.0	581.0	251.0	605.0	535.0	644.0	653.0	338.0	387.0	549.0	285.0	520.0	350.0	466.0	345.0	323.0	420.0	

In [31]:
#Remove header
#Keep chr, start, end, and the average
#Save output
! tail -n+2 union-cov-ploidy.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $29, $30}' \
> ploidy-cov-averages.bedgraph

In [32]:
#Check output: chr, start, end, average %meth
!head ploidy-cov-averages.bedgraph
!wc -l ploidy-cov-averages.bedgraph

141.66666666666666	119.66666666666667
350.0833333333333	308.1666666666667
275.75	243.5
426.9166666666667	367.25
460.3333333333333	398.25
426.5833333333333	377.25
486.5	430.0
467.6666666666667	403.6666666666667
408.9166666666667	355.25
316.25	281.0
 12794074 ploidy-cov-averages.bedgraph


## 5. Characterize methylation for each CpG dinucleotude

- methylated: ≥ 50%
- sparsely methylated: 10-50%
- unmethylated: ≤ 10%

In [21]:
#8 individual sample files + 1 union bedgraph
!find zr3644*5x.bedgraph

zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_21_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_22_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_23_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_24_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph
zr3644_2_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph

### 5a. Methylated loci

In [22]:
%%bash
for f in zr3644*5x.bedgraph
do
    awk '{if ($4 >= 50) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-Meth
done

In [23]:
!head *-Meth

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
NC_047559.1 4270 4272 50.000000
NC_047559.1 4791 4793 68.750000
NC_047559.1 4835 4837 88.888889
NC_047559.1 4843 4845 91.304348
NC_047559.1 4887 4889 80.000000
NC_047559.1 7490 7492 93.750000
NC_047559.1 7671 7673 83.333333
NC_047559.1 7814 7816 60.000000
NC_047559.1 7850 7852 62.500000
NC_047559.1 7867 7869 77.777778

==> zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
NC_047559.1 4843 4845 65.384615
NC_047559.1 4887 4889 58.333333
NC_047559.1 5316 5318 57.142857
NC_047559.1 7129 7131 96.551724
NC_047559.1 7490 7492 84.615385
NC_047559.1 7671 7673 90.909091
NC_047559.1 7814 7816 75.000000
NC_047559.1 7850 7852 75.000000
NC_047559.1 7867 7869 95.238095
NC_047559.1 7885 7887 76.470588

==> zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
NC_047559.1 4791 4793 53.125000
NC_047559.1 4835 4837 68.421053
NC_047559.1 4843 4845 68.421053
NC_047559.1 4887 4889 60.000000
NC_047559.1 76

In [24]:
!wc -l *-Meth

  743737 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  756315 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  637993 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  740056 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  677853 zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  830917 zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  693543 zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  734773 zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  707271 zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  685738 zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  619143 zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  723787 zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  719026 zr3644_21_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  675299 zr3644_22_R1_val_1_val_1_val_1_bismark_bt2_

In [28]:
#Get line counts for each fine
# Remove 10th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-Meth \
| sed '26,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-Meth-counts.txt

In [29]:
!head zr3644_5x-Meth-counts.txt

743737	zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
756315	zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
637993	zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
740056	zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
677853	zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
830917	zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
693543	zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
734773	zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
707271	zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
685738	zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth


### 5b. Sparsely methylated loci

In [30]:
%%bash
for f in zr3644*5x.bedgraph
do
    awk '{if ($4 < 50) { print $1, $2, $3, $4}}' ${f} \
    | awk '{if ($4 > 10) { print $1, $2, $3, $4 }}' \
    > ${f}-sparseMeth
done

In [31]:
!head *-sparseMeth

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
NC_001276.1 4384 4386 15.555556
NC_001276.1 4396 4398 17.073171
NC_001276.1 4402 4404 12.820513
NC_001276.1 4444 4446 15.151515
NC_001276.1 4446 4448 13.333333
NC_001276.1 4488 4490 11.764706
NC_047559.1 4397 4399 16.666667
NC_047559.1 4909 4911 28.571429
NC_047559.1 5500 5502 20.000000
NC_047559.1 7129 7131 44.444444

==> zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
NC_001276.1 4557 4559 10.526316
NC_001276.1 4573 4575 11.111111
NC_001276.1 6462 6464 11.627907
NC_047559.1 4791 4793 38.461538
NC_047559.1 4835 4837 48.148148
NC_047559.1 4909 4911 33.333333
NC_047559.1 5605 5607 30.000000
NC_047559.1 5613 5615 22.222222
NC_047559.1 7716 7718 33.333333
NC_047559.1 8171 8173 44.444444

==> zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
NC_001276.1 4485 4487 12.500000
NC_047559.1 5500 5502 28.571429
NC_047559.1 7129 7131 45.833333
NC_047559.1 7490 7492 38.888

In [32]:
!wc -l *-sparseMeth

  528855 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  558684 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  495856 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  567817 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  354184 zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  571945 zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  501880 zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  487599 zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  531718 zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  506442 zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  523037 zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  549976 zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  548000 zr3644_21_R1_val_1_val_1_val_1_bismark_bt2_p

In [33]:
#Get line counts for each fine
# Remove 10th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-sparseMeth \
| sed '26,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-sparseMeth-counts.txt

In [34]:
!head zr3644_5x-sparseMeth-counts.txt

528855	zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
558684	zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
495856	zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
567817	zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
354184	zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
571945	zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
501880	zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
487599	zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
531718	zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
506442	zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth


### 5c. Unmethylated loci

In [35]:
%%bash
for f in zr3644*5x.bedgraph
do
    awk '{if ($4 <= 10) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-unMeth
done

In [36]:
!head *-unMeth

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
NC_001276.1 34 36 3.092784
NC_001276.1 123 125 0.851064
NC_001276.1 305 307 3.314917
NC_001276.1 433 435 1.986755
NC_001276.1 457 459 1.538462
NC_001276.1 482 484 0.638978
NC_001276.1 609 611 1.479290
NC_001276.1 781 783 1.457726
NC_001276.1 826 828 1.346801
NC_001276.1 951 953 5.645161

==> zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
NC_001276.1 34 36 1.162791
NC_001276.1 123 125 0.425532
NC_001276.1 305 307 1.081081
NC_001276.1 433 435 1.167315
NC_001276.1 457 459 2.158273
NC_001276.1 482 484 2.076125
NC_001276.1 609 611 1.543210
NC_001276.1 781 783 1.298701
NC_001276.1 826 828 2.205882
NC_001276.1 951 953 1.785714

==> zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
NC_001276.1 34 36 3.061224
NC_001276.1 123 125 0.826446
NC_001276.1 305 307 3.225806
NC_001276.1 433 435 1.133144
NC_001276.1 457 459 0.529101
NC_001276.1 482 484 1.749271
NC_001276.1 609 611 2.425876

In [37]:
!wc -l *-unMeth

 5209767 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5479027 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5010195 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5373555 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 4141333 zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5572667 zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 4938951 zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5375811 zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5109927 zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5089706 zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 4999645 zr3644_1_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5323583 zr3644_20_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5230773 zr3644_21_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 4917890 zr3644_22_R1_val_

In [38]:
#Get line counts for each fine
# Remove 10th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-unMeth \
| sed '26,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-unMeth-counts.txt

In [39]:
!head zr3644_5x-unMeth-counts.txt

5209767	zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
5479027	zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
5010195	zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
5373555	zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
4141333	zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
5572667	zr3644_15_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
4938951	zr3644_16_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
5375811	zr3644_17_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
5109927	zr3644_18_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
5089706	zr3644_19_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth


## 6. Characterize genomic location of CpGs

I will identify overlaps between CpG loci (methylated, sparsely methylated, unmethylated) and various genome feature tracks:

- gene
- exon UTR
- CDS
- intron
- upstream flanks
- downstream flanks
- intergenic regions
- lncRNA
- transposable elements

Since the exon track = exon UTR + CDS, and mRNA = exon + intron, I will not need to use those tracks separately.

### 6a. Create BEDfiles

In [40]:
#25 file types (24 samples + 1 union), 3 files per type (Meth, sparseMeth, unMeth) = 75 total
!find zr3644*5x.bedgraph-*
!find zr3644*5x.bedgraph-* | wc -l

zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
zr3644_14_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth


In [41]:
%%bash

for f in zr3644*5x.bedgraph-*
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

  743737 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  528855 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
 5209767 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
  756315 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  558684 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
 5479027 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
  637993 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  495856 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
 5010195 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
  740056 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  567817 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
 5373555 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
  677853 zr3644_14_R1_val_1_val_1_val_1_bism

In [6]:
#Convert 1x bedgraph (DSS CpG background) to BEDfile. 
#Remove header
#Save output: Will intersect with genes to get information important for annotation
!awk '{print $1"\t"$2"\t"$3}' union_1x.bedgraph \
| tail -n+2 \
> union_1x.bedgraph.bed
!head union_1x.bedgraph.bed

NC_001276.1	34	36
NC_001276.1	123	125
NC_001276.1	305	307
NC_001276.1	433	435
NC_001276.1	457	459
NC_001276.1	482	484
NC_001276.1	609	611
NC_001276.1	781	783
NC_001276.1	826	828
NC_001276.1	951	953


### 6b. Gene

In [42]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_gene.gff \
    > ${f}-Gene
done

In [43]:
#Check output
!head *Gene

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-Gene <==
NC_047559.1	356335	356337
NC_047559.1	356436	356438
NC_047559.1	356685	356687
NC_047559.1	357799	357801
NC_047559.1	357900	357902
NC_047559.1	357955	357957
NC_047559.1	358313	358315
NC_047559.1	362609	362611
NC_047559.1	416078	416080
NC_047559.1	416920	416922

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-Gene <==
NC_047559.1	10270	10272
NC_047559.1	10292	10294
NC_047559.1	10358	10360
NC_047559.1	18257	18259
NC_047559.1	61081	61083
NC_047559.1	62770	62772
NC_047559.1	68070	68072
NC_047559.1	72245	72247
NC_047559.1	81263	81265
NC_047559.1	100249	100251

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-Gene <==
NC_047559.1	10314	10316
NC_047559.1	10845	10847
NC_047559.1	10859	10861
NC_047559.1	10899	10901
NC_047559.1	10920	10922
NC_047559.1	10950	10952
NC_047559.1	11000	11002
NC_047559.1	11026	11028
NC_047559.1	14214	14216
NC_047559.1	14232	14234

==> 

In [44]:
#Count number of overlaps
!wc -l *Gene

  704609 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-Gene
  375370 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-Gene
 2973037 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-Gene
  715222 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-Gene
  396962 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-Gene
 3102569 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-Gene
  603534 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-Gene
  357468 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-Gene
 2835308 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-Gene
  700738 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-Gene
  409307 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-Gene
 3040647 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-

In [46]:
#Get line counts for each fine
# Remove 76th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-Gene \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-Gene-counts.txt

In [8]:
#Characterize overlap for downstream annotation
!{bedtoolsDirectory}intersectBed \
-wb \
-a union_1x.bedgraph.bed \
-b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_gene.gff \
> union_1x-Gene-wb.bed
!head union_1x-Gene-wb.bed

NC_047559.1	10155	10157	NC_047559.1	Gnomon	gene	9839	11386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	10215	10217	NC_047559.1	Gnomon	gene	9839	11386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	10270	10272	NC_047559.1	Gnomon	gene	9839	11386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	10292	10294	NC_047559.1	Gnomon	gene	9839	11386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	10314	10316	NC_047559.1	Gnomon	gene	9839	11386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	10358	10360	NC_047559.1	Gnomon	gene	9839	11386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=

### 6c. CDS

In [6]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_CDS.gff \
    > ${f}-CDS
done

In [7]:
#Check output
!head *CDS

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-CDS <==
NC_047559.1	433271	433273
NC_047559.1	545968	545970
NC_047559.1	548186	548188
NC_047559.1	548188	548190
NC_047559.1	548918	548920
NC_047559.1	548941	548943
NC_047559.1	548958	548960
NC_047559.1	549827	549829
NC_047559.1	549832	549834
NC_047559.1	551729	551731

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-CDS <==
NC_047559.1	123636	123638
NC_047559.1	123655	123657
NC_047559.1	139297	139299
NC_047559.1	139334	139336
NC_047559.1	139383	139385
NC_047559.1	139389	139391
NC_047559.1	139396	139398
NC_047559.1	139430	139432
NC_047559.1	139438	139440
NC_047559.1	139443	139445

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-CDS <==
NC_047559.1	14862	14864
NC_047559.1	15162	15164
NC_047559.1	15194	15196
NC_047559.1	15204	15206
NC_047559.1	15291	15293
NC_047559.1	15293	15295
NC_047559.1	15336	15338
NC_047559.1	15338	15340
NC_047559.1	15357	15359
NC_047559.1	15

In [8]:
#Count number of overlaps
!wc -l *CDS

  303330 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-CDS
   63358 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-CDS
  754584 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-CDS
  304639 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-CDS
   67810 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-CDS
  769482 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-CDS
  241585 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-CDS
   59366 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-CDS
  686708 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-CDS
  298469 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-CDS
   69735 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-CDS
  755670 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-

In [9]:
#Get line counts for each fine
# Remove 76th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-CDS \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-CDS-counts.txt

### 6d. Exon UTR

In [47]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_exonUTR.gff \
    > ${f}-exonUTR
done

In [48]:
#Check output
!head *exonUTR

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-exonUTR <==
NC_047559.1	416078	416080
NC_047559.1	418539	418541
NC_047559.1	544784	544786
NC_047559.1	571583	571585
NC_047559.1	571793	571795
NC_047559.1	571838	571840
NC_047559.1	571906	571908
NC_047559.1	571929	571931
NC_047559.1	572049	572051
NC_047559.1	572075	572077

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-exonUTR <==
NC_047559.1	416052	416054
NC_047559.1	416071	416073
NC_047559.1	416088	416090
NC_047559.1	431124	431126
NC_047559.1	469758	469760
NC_047559.1	545687	545689
NC_047559.1	560332	560334
NC_047559.1	572153	572155
NC_047559.1	572900	572902
NC_047559.1	597401	597403

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-exonUTR <==
NC_047559.1	10899	10901
NC_047559.1	10920	10922
NC_047559.1	10950	10952
NC_047559.1	11000	11002
NC_047559.1	11026	11028
NC_047559.1	14214	14216
NC_047559.1	14232	14234
NC_047559.1	14243	14245
NC_047559.1	14259	14261
NC

In [49]:
#Count number of overlaps
!wc -l *exonUTR

   41929 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-exonUTR
   30046 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-exonUTR
  366136 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-exonUTR
   42629 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-exonUTR
   31927 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-exonUTR
  379558 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-exonUTR
   39179 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-exonUTR
   30144 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-exonUTR
  355660 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-exonUTR
   42282 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-exonUTR
   33224 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-exonUTR
  375672 zr3644_13_R1_val_1_val_1_v

In [50]:
#Get line counts for each fine
# Remove 28th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-exonUTR \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-exonUTR-counts.txt

### 6e. Intron

In [51]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_intron.bed \
    > ${f}-intron
done

In [52]:
#Check output
!head *intron

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intron <==
NC_047559.1	356335	356337
NC_047559.1	356436	356438
NC_047559.1	356685	356687
NC_047559.1	357799	357801
NC_047559.1	357900	357902
NC_047559.1	357955	357957
NC_047559.1	358313	358315
NC_047559.1	362609	362611
NC_047559.1	416920	416922
NC_047559.1	418551	418553

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intron <==
NC_047559.1	10270	10272
NC_047559.1	10292	10294
NC_047559.1	10358	10360
NC_047559.1	18257	18259
NC_047559.1	61081	61083
NC_047559.1	62770	62772
NC_047559.1	68070	68072
NC_047559.1	72245	72247
NC_047559.1	81263	81265
NC_047559.1	100249	100251

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-intron <==
NC_047559.1	10314	10316
NC_047559.1	10845	10847
NC_047559.1	10859	10861
NC_047559.1	14770	14772
NC_047559.1	14790	14792
NC_047559.1	15481	15483
NC_047559.1	15489	15491
NC_047559.1	15538	15540
NC_047559.1	18064	18066
NC_047559.1	18078	18080

In [53]:
#Count number of overlaps
!wc -l *intron

  361376 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intron
  282419 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intron
 1858030 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-intron
  369981 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intron
  297780 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intron
 1959575 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-intron
  324571 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intron
  268438 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intron
 1798371 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-intron
  362006 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intron
  306884 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intron
 1915185 zr3644_13_R1_val_1_val_1_val_1_bismar

In [54]:
#Get line counts for each fine
# Remove 76th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-intron \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-intron-counts.txt

### 6f. Upstream flanks

In [75]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_upstream.gff \
    > ${f}-upstreamFlanks
done

In [76]:
#Check output
!head *upstreamFlanks

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-upstreamFlanks <==
NC_047559.1	576634	576636
NC_047559.1	576693	576695
NC_047559.1	576752	576754
NC_047559.1	1800772	1800774
NC_047559.1	1801087	1801089
NC_047559.1	1801099	1801101
NC_047559.1	1801111	1801113
NC_047559.1	1801125	1801127
NC_047559.1	3158461	3158463
NC_047559.1	3907322	3907324

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-upstreamFlanks <==
NC_047559.1	8979	8981
NC_047559.1	9122	9124
NC_047559.1	9140	9142
NC_047559.1	142484	142486
NC_047559.1	335878	335880
NC_047559.1	335886	335888
NC_047559.1	335892	335894
NC_047559.1	335912	335914
NC_047559.1	335932	335934
NC_047559.1	778316	778318

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-upstreamFlanks <==
NC_047559.1	8970	8972
NC_047559.1	9159	9161
NC_047559.1	9237	9239
NC_047559.1	9240	9242
NC_047559.1	9774	9776
NC_047559.1	9781	9783
NC_047559.1	9787	9789
NC_047559.1	9795	9797
NC_047559.1	9803	98

In [77]:
#Count number of overlaps
!wc -l *upstreamFlanks

    3277 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-upstreamFlanks
   13144 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-upstreamFlanks
  281363 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-upstreamFlanks
    3423 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-upstreamFlanks
   13735 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-upstreamFlanks
  296613 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-upstreamFlanks
    2942 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-upstreamFlanks
   12111 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-upstreamFlanks
  275129 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-upstreamFlanks
    3314 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-upstreamFlanks
   13439 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._

In [78]:
#Get line counts for each fine
# Remove 28th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-upstreamFlanks \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-upstreamFlanks-counts.txt

### 6g. Downstream flanks

In [59]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_downstream.gff \
    > ${f}-downstreamFlanks
done

In [60]:
#Check output
!head *downstreamFlanks

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-downstreamFlanks <==
NC_047559.1	344549	344551
NC_047559.1	344803	344805
NC_047559.1	344812	344814
NC_047559.1	344972	344974
NC_047559.1	344976	344978
NC_047559.1	434252	434254
NC_047559.1	434266	434268
NC_047559.1	576634	576636
NC_047559.1	576693	576695
NC_047559.1	576752	576754

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-downstreamFlanks <==
NC_047559.1	142484	142486
NC_047559.1	142531	142533
NC_047559.1	142545	142547
NC_047559.1	142593	142595
NC_047559.1	258248	258250
NC_047559.1	325786	325788
NC_047559.1	325842	325844
NC_047559.1	344623	344625
NC_047559.1	344785	344787
NC_047559.1	344794	344796

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-downstreamFlanks <==
NC_047559.1	16061	16063
NC_047559.1	16105	16107
NC_047559.1	16112	16114
NC_047559.1	16220	16222
NC_047559.1	16260	16262
NC_047559.1	16289	16291
NC_047559.1	16310	16312
NC_047559.1	16369	16371

In [61]:
#Count number of overlaps
!wc -l *downstreamFlanks

   14259 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-downstreamFlanks
   22064 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-downstreamFlanks
  229754 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-downstreamFlanks
   15052 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-downstreamFlanks
   23547 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-downstreamFlanks
  244273 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-downstreamFlanks
   13711 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-downstreamFlanks
   20189 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-downstreamFlanks
  228132 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-downstreamFlanks
   14438 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-downstreamFlanks
   23164 zr3644_13_R1_val_1_val_1_va

In [62]:
#Get line counts for each fine
# Remove 28th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-downstreamFlanks \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-downstreamFlanks-counts.txt

### 6h. Intergenic regions

In [63]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_intergenic.bed \
    > ${f}-intergenic
done

In [64]:
#Check output
!head *intergenic

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intergenic <==
NC_047559.1	4270	4272
NC_047559.1	4791	4793
NC_047559.1	4835	4837
NC_047559.1	4843	4845
NC_047559.1	4887	4889
NC_047559.1	7490	7492
NC_047559.1	7671	7673
NC_047559.1	7814	7816
NC_047559.1	7850	7852
NC_047559.1	7867	7869

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intergenic <==
NC_047559.1	4397	4399
NC_047559.1	4909	4911
NC_047559.1	5500	5502
NC_047559.1	7129	7131
NC_047559.1	21982	21984
NC_047559.1	23360	23362
NC_047559.1	23610	23612
NC_047559.1	25334	25336
NC_047559.1	26448	26450
NC_047559.1	26699	26701

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-intergenic <==
NC_047559.1	20319	20321
NC_047559.1	20341	20343
NC_047559.1	20363	20365
NC_047559.1	20385	20387
NC_047559.1	20407	20409
NC_047559.1	20429	20431
NC_047559.1	20451	20453
NC_047559.1	20924	20926
NC_047559.1	20946	20948
NC_047559.1	20954	20956

==> zr3644_11_R1_val_1_val_1_val_1_b

In [65]:
#Count number of overlaps
!wc -l *intergenic

   22733 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intergenic
  119886 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intergenic
 1748992 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-intergenic
   23815 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intergenic
  126168 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intergenic
 1860431 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-intergenic
   18892 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intergenic
  107574 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intergenic
 1694738 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-intergenic
   22700 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-intergenic
  123761 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-intergenic
 1

In [66]:
#Get line counts for each fine
# Remove 28th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-intergenic \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-intergenic-counts.txt

### 6i. lncRNA

In [67]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_lncRNA.gff \
    > ${f}-lncRNA
done

In [68]:
#Check output
!head *lncRNA

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-lncRNA <==
NC_047559.1	416078	416080
NC_047559.1	416920	416922
NC_047559.1	418539	418541
NC_047559.1	418551	418553
NC_047559.1	418582	418584
NC_047559.1	418592	418594
NC_047559.1	1664745	1664747
NC_047559.1	1664773	1664775
NC_047559.1	1688433	1688435
NC_047559.1	1688466	1688468

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-lncRNA <==
NC_047559.1	10270	10272
NC_047559.1	10292	10294
NC_047559.1	10358	10360
NC_047559.1	242614	242616
NC_047559.1	258015	258017
NC_047559.1	416052	416054
NC_047559.1	416071	416073
NC_047559.1	416088	416090
NC_047559.1	419514	419516
NC_047559.1	419784	419786

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-lncRNA <==
NC_047559.1	10314	10316
NC_047559.1	10845	10847
NC_047559.1	10859	10861
NC_047559.1	10899	10901
NC_047559.1	10920	10922
NC_047559.1	10950	10952
NC_047559.1	11000	11002
NC_047559.1	11026	11028
NC_047559.1	226734	226736
N

In [69]:
#Count number of overlaps
!wc -l *lncRNA

   11815 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-lncRNA
   19718 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-lncRNA
  167405 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-lncRNA
   11896 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-lncRNA
   20834 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-lncRNA
  176096 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-lncRNA
   10290 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-lncRNA
   18038 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-lncRNA
  163933 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-lncRNA
   11791 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-lncRNA
   21620 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-lncRNA
  173954 zr3644_13_R1_val_1_val_1_val_1_bismar

In [70]:
#Get line counts for each fine
# Remove 28th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-lncRNA \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-lncRNA-counts.txt

### 6j. Transposable elements

In [71]:
%%bash
for f in *bed
do
    /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
    -u \
    -a ${f} \
    -b /Volumes/web-1/halfshell/genomic-databank/cgigas_uk_roslin_v1_rm.te.bed \
    > ${f}-TE
done

In [72]:
#Check output
!head *TE

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-TE <==
NC_047559.1	344549	344551
NC_047559.1	344803	344805
NC_047559.1	344812	344814
NC_047559.1	356335	356337
NC_047559.1	356436	356438
NC_047559.1	356685	356687
NC_047559.1	357799	357801
NC_047559.1	357900	357902
NC_047559.1	357955	357957
NC_047559.1	358313	358315

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-TE <==
NC_047559.1	26448	26450
NC_047559.1	26699	26701
NC_047559.1	42362	42364
NC_047559.1	42415	42417
NC_047559.1	43952	43954
NC_047559.1	51213	51215
NC_047559.1	51242	51244
NC_047559.1	51249	51251
NC_047559.1	51298	51300
NC_047559.1	51319	51321

==> zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-TE <==
NC_047559.1	15746	15748
NC_047559.1	26419	26421
NC_047559.1	26463	26465
NC_047559.1	26485	26487
NC_047559.1	26509	26511
NC_047559.1	26532	26534
NC_047559.1	26547	26549
NC_047559.1	26555	26557
NC_047559.1	26561	26563
NC_047559.1	26573	26575

==> zr3644_1

In [73]:
#Count number of overlaps
!wc -l *TE

  189566 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-TE
  251548 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-TE
 1637275 zr3644_10_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-TE
  198940 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-TE
  266854 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-TE
 1744645 zr3644_11_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-TE
  169747 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-TE
  227639 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-TE
 1583872 zr3644_12_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-TE
  191696 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-TE
  258316 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-TE
 1704139 zr3644_13_R1_val_1_val_1_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-TE
  150122

In [74]:
#Get line counts for each fine
# Remove 28th line (total entries)
#Ensure output is tab-delimited
#Save output
!wc -l *-TE \
| sed '76,$ d' \
| awk '{print $1"\t"$2}' \
> zr3644_5x-TE-counts.txt